In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pyproj import Proj, transform

plt.rcParams['font.family'] = 'Malgun Gothic'

In [51]:
#기본 정보
coord_std=pd.read_excel('./기본정보_위경도.xlsx')

#숙박 유형
pension=pd.read_excel("./숙박유형_관광펜션업.xlsx")
camping=pd.read_excel("./숙박유형_일반야영장업.xlsx")
hanok=pd.read_excel("./숙박유형_한옥체험업.xlsx")
resort=pd.read_excel("./숙박유형_호텔리조트.xlsx") 

#비대면
alien1=pd.read_csv('./비대면_외지인방문자수_1.csv',encoding='cp949')
alien2=pd.read_csv('./비대면_외지인방문자수_2.csv',encoding='cp949')
resider1=pd.read_csv('./비대면_현지인방문자수_1.csv',encoding='cp949')
resider2=pd.read_csv('./비대면_현지인방문자수_2.csv',encoding='cp949')

#식음및부대시설
meal=pd.read_csv('./부대시설_음식점.csv',encoding='cp949')
cafe=pd.read_csv('./부대시설_휴게음식점.csv',encoding='utf-8')

#콘텐츠
contents1=pd.read_csv('./콘텐츠_관광지_검색순위_상.csv',encoding='cp949')
contents2=pd.read_csv('./콘텐츠_관광지_검색순위_하.csv',encoding='cp949')

숙박유형

In [52]:
def accomm_preprocess(accomm):
    accomm.drop(accomm[(accomm['영업상태구분코드']!=1)].index,inplace=True)
    accomm=accomm[accomm['개방자치단체코드'].between(4800000,5010000)]
    accomm=accomm.loc[:,['개방자치단체코드','좌표정보(X)','좌표정보(Y)']]
    return accomm

ps_1=accomm_preprocess(pension)
cp_1=accomm_preprocess(camping)
hk_1=accomm_preprocess(hanok)
rs_1=accomm_preprocess(resort)

In [53]:
proj_1 = Proj(init='epsg:2097')
proj_2 = Proj(init='epsg:4326')

def coordinate_transform(acco):
    convert = transform(proj_1, proj_2, acco['좌표정보(X)'].values, acco['좌표정보(Y)'].values)
    acco['경도'] = convert[0]
    acco['위도'] = convert[1]
    return acco

ps=coordinate_transform(ps_1)
cp=coordinate_transform(cp_1)
hk=coordinate_transform(hk_1)
rs=coordinate_transform(rs_1)

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\user\AppData\Local\Temp\ipykernel_135128\985871408.py:5: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas

In [ ]:
accomm_df=pd.concat([ps,cp,hk,rs],ignore_index=True)
accomm_df.drop(['좌표정보(X)','좌표정보(Y)'],axis=1,inplace=True)

In [54]:
accomm_num=accomm_df.dropna(axis=0)
acco_n=accomm_num['개방자치단체코드'].value_counts() #지역별 숙소 개수
print(len(acco_n))

22


부대시설

In [70]:
#카페
def cafe_preprocess(cafe):
    cafe.drop(cafe[(cafe['영업상태명']!='영업/정상')].index,inplace=True)
    cafe=cafe.loc[:,['소재지전체주소','좌표정보(X)','좌표정보(Y)']]
    cafe.dropna(axis=0,inplace=True)
    return cafe

cafe_df=cafe_preprocess(cafe)
#cafe_regions=cafe['소재지전체주소'].copy()

# c_r=[]
# for i in range(len(cafe_regions)):
#     c_r.append(cafe_regions[i][4])

In [79]:
cafe_address=cafe_df['소재지전체주소']
cafe_address_fr=cafe_address.str.slice(start=4, stop=9, step=1)
cafe_n=cafe_address_fr.value_counts()
print(cafe_n)

소재지전체주소
 순천시     1226
 여수시     1179
 목포시      860
 광양시      563
 나주시      416
 무안군      366
 담양군      264
 화순군      261
 해남군      260
 완도군      193
 영암군      167
 고흥군      154
 장성군      146
 영광군      139
 장흥군      136
 강진군      121
 보성군      120
 구례군      112
 곡성군       96
 함평군       85
 진도군       83
 신안군       61
Name: count, dtype: int64


In [78]:
accomm_num=accomm_df.dropna(axis=0)
acco_n=accomm_num['개방자치단체코드'].value_counts() #지역별 숙소 개수
print(acco_n)

개방자치단체코드
4810000    268
4820000    143
4890000     70
4830000     52
4930000     44
4940000     43
4800000     39
4920000     37
4870000     28
4850000     27
4980000     27
5010000     27
4880000     26
4950000     25
4860000     20
4910000     17
4970000     17
4990000     17
4900000     13
4840000     11
4960000      9
5000000      7
Name: count, dtype: int64


매력 요소: 비대면 지수, 외부 유입 지수

In [ ]:
alien=pd.concat([alien1,alien2],axis=1,ignore_index=True)
alien.drop([2],axis=1,inplace=True) #순위 제거
alien['평균외지인방문자수']=(alien[1]+alien[3])/2 

resider=pd.concat([resider1,resider2],axis=1,ignore_index=True)
resider.drop([2],axis=1,inplace=True)
resider['평균현지인방문자수']=(resider[1]+resider[3])/2

regions=alien[0]

calm_score=alien['평균외지인방문자수']/resider['평균현지인방문자수']*100 #선행연구 참고
induce_score=alien['평균외지인방문자수']-resider['평균현지인방문자수']

attract_df=pd.concat([regions,calm_score,induce_score],axis=1)
attract_col=['지역','비대면지수','외부유입지수']
attract_df.columns=attract_col
attract_df.set_index('지역',inplace=True)

In [ ]:
acco_n_fr=list(acco_n).copy()
cafe_n_fr=list(cafe_fr).copy()
cafe_n_fr
# attract_df['숙소개수']=acco_n_fr
# attract_df['카페개수']=cafe_n_fr

# print(attract_df)

# attract_df.corr() #둘 다 0.717266
# sns.pairplot(attract_df) #조용한 곳일수록 유입이 많다.

콘텐츠

In [ ]:
contents2.drop('광역시/도',axis=1,inplace=True)

In [ ]:
count_num=0
sites=[]
rnum=[]

for i in range(len(contents2)): #22'09~23'08
   for j in range(len(contents1)): #21'09~22'08
        if contents2['관광지명'][i]==contents1['관광지명'][j]:
            count_num+=1
            sites.append(contents2['관광지명'][i])
            rnum.append(contents2['검색건수'][i])

print(count_num) #451: 인기 관광지 500곳 중 451곳은 반복해서 수요가 많다.
print(sites) #사람들이 반복해서 찾는 곳

In [ ]:
x1=np.arange(len(contents2))
plt.scatter(x1,contents2['검색건수'])
x2=np.arange(len(rnum))
plt.scatter(x2,rnum,c='r')

plt.ylabel('검색건수')
plt.title('전라남도 관광지 검색량') #가는 곳(만) 또 감

In [ ]:
ct2=contents2.drop_duplicates(subset=['시/군/구'])
contents_df=ct2.drop(['중분류 카테고리','순위','관광지명','도로명주소'], axis=1).reset_index(drop=True)
print(contents_df)

데이터 취합

In [ ]:
cafe_final=cafe_fr.reset_index()
cafe_final.columns=['지역','카페 개수']

diff=meal['항목'].compare(cafe_final['지역'])
print(diff)

print(cafe_final)

In [ ]:
cat=['여수시', '순천시', '목포시', '광양시', '나주시', '무안군', '해남군', '영암군', 
     '고흥군', '화순군', '완도군', '담양군', '영광군', '보성군', '장성군', '장흥군', 
     '강진군', '구례군', '신안군', '진도군', '곡성군', '함평군']
cafe_num=[]

for i in range(len(cat)):
    for j in range(len(cafe_final)):
        if cat[i]==cafe_final['지역'][j]:
            cafe_num.append(cafe_final['카페 개수'][j])
print(cafe_num)

print(cafe_final['지역'])

In [ ]:
visit=[]

for i in range(len(meal)):
    for j in range(len(contents_df)):
        if meal['항목'][i]==contents_df['시/군/구'][j]:
            visit.append(contents_df['검색건수'][j])

print(visit)

cafes=[]

for i in range(len(meal)):
    for j in range(len(cafe_final)):
        if meal['항목'][i]==cafe_final['지역'][j]:
            cafes.append(cafe_final['카페 개수'][i])

print(cafes)


In [ ]:
visit_num=pd.DataFrame(visit)
meal['검색건수']=visit

meal_columns=['항목','음식점개수','경도','위도','검색건수']
meal.columns=meal_columns
meal.loc[0,'음식점개수']=3875
meal.loc[1,'음식점개수']=3157
meal.loc[2,'음식점개수']=2977
meal.loc[3,'음식점개수']=1974
meal.loc[4,'음식점개수']=1343
meal.loc[5,'음식점개수']=1016

print(meal)

In [ ]:
sites_num=pd.DataFrame([76,41,40,26,25,19,25,12,16,23,18,
                        23,21,16,18,13,17,18,14,16,11,12])
tour_df=pd.concat([meal,sites_num],axis=1,ignore_index=True)
tour_col=['지역','음식점개수','경도','위도','검색건수','관광지개수']
tour_df.columns=tour_col
tour_df.set_index('지역',inplace=True)
tour_df.info()
attract_df.info()

In [ ]:
a_index=attract_df.index
t_index=tour_df.index
calmness=[]
entranger=[]
accommodation=[]

for i in range(len(tour_df)):
    for j in range(len(attract_df)):
        if t_index[i]==a_index[j]:
            calmness.append(attract_df['비대면지수'][j])
            entranger.append(attract_df['외부유입지수'][j])
            accommodation.append(attract_df['숙소개수'][j])
        else:
            continue


tour_df['비대면지수']=calmness.copy()
tour_df['외부인유입지수']=entranger.copy()
tour_df['숙소개수']=accommodation.copy()

print(tour_df)

In [ ]:
tour_corr=tour_df.corr()
print(tour_corr)

sns.pairplot(tour_corr)

#외부유입지수<-> (특히)음식점개수, 검색건수, 관광지개수
#-> 외지인은 덜 유명한 곳을 찾아간다?
#외부유입지수와 비대면지수는 상생

시각화

In [ ]:
xmin=min(coord_std['경도'])
xmax=max(coord_std['경도'])
ymin=min(coord_std['위도'])
ymax=max(coord_std['위도'])

In [ ]:
#숙소유형
plt.scatter(x=accomm_df['경도'],y=accomm_df['위도'],alpha=0.2,c='b',label='숙소')

plt.scatter(x=cafe['경도'],y=cafe['위도'],alpha=0.2,c='m',label='카페')

plt.scatter(x=tour_df['경도'],y=tour_df['위도'],c='g',label='지역')


plt.title('숙소 및 카페 분포도')

plt.xlim(xmin,xmax)
plt.ylim(ymin,ymax)

plt.legend()

plt.show()

In [ ]:
# from sklearn.cluster import KMeans

# def doKmeans(X, nclust=2):
#     model = KMeans(nclust)
#     model.fit(X)
#     clust_labels = model.predict(X)
#     cent = model.cluster_centers_
#     return (clust_labels, cent)

# clust_labels, cent = doKmeans(tour_df, 2)
# kmeans = pd.DataFrame(clust_labels)
# tour_df.insert((tour_df[1]),'kmeans',kmeans)

# #Plot the clusters obtained using k means
# fig = plt.figure()
# ax = fig.add_subplot(111)
# scatter = ax.scatter(tour_df['외부인유입지수'],tour_df['비대면지수'],
#                      c=kmeans[0],s=50)
# ax.set_title('K-Means Clustering')
# ax.set_xlabel('GDP per Capita')
# ax.set_ylabel('Corruption')
# plt.colorbar(scatter)